In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib as pl
import xarray as xa

# Spatial data for the model

In [ ]:
nc_path = pl.Path("../synthetic-valley/data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)
lake_location = nc_ds["lake_location"].to_numpy()

In [ ]:
nc_ds

In [ ]:
name = "sv"

In [ ]:
# load the simple model setup
existing_ws = pl.Path("../models/synthetic-valley-vg")
ws = existing_ws.parent / "synthetic-valley-vg-prt"

## Load the existing model

In [ ]:
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=existing_ws, write_headers=False
)

In [ ]:
nper = sim.tdis.nper.array
gwf = sim.get_model(name)
nlay, ncpl = gwf.disv.nlay.array, gwf.disv.ncpl.array
cell2d = gwf.disv.cell2d.array
vertices = gwf.disv.vertices.array
shape3d = (nlay, ncpl)

In [ ]:
perlen = sim.tdis.perioddata.array["perlen"]
tracktimes = np.cumsum(perlen[1:]) + perlen[0]
tracktimes = np.insert(tracktimes, 0, 0)

### Change the simulation workspace

In [ ]:
sim.set_sim_path(ws)

### Get the gwf nodes the lake is connected to

In [ ]:
pak = gwf.get_package("LAK-1")

In [ ]:
lak_conn = pak.connectiondata.array["cellid"]

In [ ]:
lak_conn

### Get the sfr connections

In [ ]:
pak = gwf.get_package("SFR-1")
sfr_conn = pak.packagedata.array["cellid"]

In [ ]:
sfr_conn

### Get the well locations

In [ ]:
pak = gwf.get_package("WEL-1")
well_loc = pak.stress_period_data.get_data(1)["cellid"]

In [ ]:
well_loc

### Create a zone array 

A different zone number will be specified for cells with lake, sfr, and well cells. This makes it easier to determine where particles terminate.


In [ ]:
izone = np.zeros(shape3d, dtype=int)
for cellid in lak_conn:
    izone[cellid] = 1
for cellid in sfr_conn:
    izone[cellid] = 2
for cellid in well_loc:
    izone[cellid] = 3

## Add PRT model

In [ ]:
# create the PRT model

prt_name = f"{name}_prt"
prt = flopy.mf6.ModflowPrt(
    sim,
    modelname=prt_name,
)

In [ ]:
# create PRT dis package
prt_dis = flopy.mf6.ModflowPrtdisv(
    prt,
    nlay=nlay,
    ncpl=ncpl,
    nvert=len(vertices),
    top=gwf.disv.top.array,
    botm=gwf.disv.botm.array,
    idomain=gwf.disv.idomain.array,
    cell2d=cell2d,
    vertices=vertices,
)

In [ ]:
# create PRT model input package
# set the porosity to gwf sy

mip = flopy.mf6.ModflowPrtmip(
    prt,
    pname="mip",
    porosity=gwf.sto.sy.array,
    izone=izone,
)

In [ ]:
# recharge release locations

top = gwf.disv.top.array
botm = gwf.disv.botm.array
xc, yc = (
    gwf.modelgrid.get_xcellcenters_for_layer(0),
    gwf.modelgrid.get_ycellcenters_for_layer(0),
)

xmin, xmax, ymin, ymax = gwf.modelgrid.extent
rch_release_loc = []
idx = 0
for node, (x, y) in enumerate(zip(xc, yc)):
    # skip nodes with centroids on the edge of the model
    if np.allclose([x], [xmin]) or np.allclose([x], [xmax]):
        continue
    if np.allclose([y], [ymin]) or np.allclose([y], [ymax]):
        continue

    x, y = int(x), int(y)
    z = float(top[node])
    cellid = (0, node)

    # # skip lake cells
    # if cellid in lak_conn.tolist():
    #     continue
    if cellid in lak_conn.tolist():
        tag = "lake"
    else:
        tag = "land"

    rch_release_loc.append((idx, cellid, float(x), float(y), z, tag))
    idx += 1

len(rch_release_loc), rch_release_loc[:10]

In [ ]:
dt = perlen[0] / (365.25 * 12.0)
nt = int(perlen[0] / dt)

tracktimes0 = [float(dt * idx) for idx in range(nt + 1)]
tracktimes1 = [perlen[0]] + (perlen[0] + np.cumsum(perlen[1:])).tolist()
perlen[0] / 365.25, tracktimes0[nt] / 365.25, tracktimes1[-1] / 365.25

In [ ]:
trackcsvfile_record_rch = f"{prt_name}_rch.trk.csv"
prp_rch = flopy.mf6.ModflowPrtprp(
    prt,
    pname="prp_rch",
    filename=f"{prt_name}.rch.prp",
    trackcsv_filerecord=trackcsvfile_record_rch,
    drape=True,
    boundnames=True,
    dry_tracking_method="DROP",
    nreleasepts=len(rch_release_loc),
    packagedata=rch_release_loc,
    perioddata={1: ["first"], 2: []},
    exit_solve_tolerance=1e-5,
    extend_tracking=True,
)

In [ ]:
budgetfile_record = f"{prt_name}.cbb"
prt_oc = flopy.mf6.ModflowPrtoc(
    prt,
    pname=f"{prt_name}-oc",
    budget_filerecord=budgetfile_record,
    ntracktimes=len(tracktimes1),
    tracktimes=[(t,) for t in tracktimes1],
    saverecord=[("BUDGET", "ALL")],
)

In [ ]:
ems = flopy.mf6.ModflowEms(
    sim,
    pname="ems",
    filename=f"{prt_name}.ems",
)
sim.register_solution_package(ems, [prt.name])

In [ ]:
exg = flopy.mf6.ModflowGwfprt(
    sim,
    exgtype="GWF6-PRT6",
    exgmnamea=gwf.name,
    exgmnameb=prt_name,
    filename=f"{gwf.name}.gwfprt",
)

## Write and run simulation

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Plot the results

### Forward particle tracks

In [ ]:
pathlines = pd.read_csv(ws / trackcsvfile_record_rch)
pathlines

In [ ]:
term_pts = pathlines[pathlines.ireason == 1]
term_pts.t.describe()

In [ ]:
pathlines["izone"].unique(), pathlines["ilay"].unique()

In [ ]:
layer_colors = ["blue", "cyan", "green", "yellow", "red"]

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 6), dpi=150)
    ax.set_aspect("equal")
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    mm.plot_grid(lw=0.5, color="0.75")
    for k in range(nlay):
        df = pathlines[pathlines["ilay"] == k + 1]
        if len(df) > 0:
            mm.plot_pathline(df, layer="all", colors=layer_colors[k], linewidth=0.1)

In [ ]:
pathlines

In [ ]:
well_irpt = pathlines[(pathlines["istatus"] == 5) & (pathlines["izone"] == 3)][
    "irpt"
].unique()
df_well = pathlines[pathlines["irpt"].isin(well_irpt)]
df_well

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 6), dpi=150)
    ax.set_aspect("equal")
    ax.set_title("Well Capture Zones")
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    mm.plot_grid(lw=0.5, color="0.75")
    for k in range(nlay):
        df = df_well[df_well["ilay"] == k + 1]
        if len(df) > 0:
            mm.plot_pathline(df, layer="all", colors=layer_colors[k], linewidth=0.1)

In [ ]:
sfr_irpt = pathlines[(pathlines["istatus"] == 2) & (pathlines["izone"] == 2)][
    "irpt"
].unique()
df_sfr = pathlines[pathlines["irpt"].isin(sfr_irpt)]
df_sfr

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 6), dpi=150)
    ax.set_aspect("equal")
    ax.set_title("Stream Capture Zones")
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    mm.plot_grid(lw=0.5, color="0.75")
    for k in range(nlay):
        df = df_sfr[df_sfr["ilay"] == k + 1]
        if len(df) > 0:
            mm.plot_pathline(df, layer="all", colors=layer_colors[k], linewidth=0.1)

In [ ]:
lak_irpt = pathlines[(pathlines["istatus"] == 2) & (pathlines["izone"] == 1)][
    "irpt"
].unique()
df_lak = pathlines[pathlines["irpt"].isin(lak_irpt)]
df_lak

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 6), dpi=150)
    ax.set_title("Lake Capture Zones")
    ax.set_aspect("equal")
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    mm.plot_grid(lw=0.5, color="0.75")
    for k in range(nlay):
        df = df_lak[df_lak["ilay"] == k + 1]
        if len(df) > 0:
            mm.plot_pathline(df, layer="all", colors=layer_colors[k], linewidth=0.1)